# GP test problem generation
A notebook whose purpose is to generate repeats of a test problem. Use the notebook `gp-test-problem.ipynb` to select good hyperparameters, then use this notebook to generate many instances of problems with these hyperparameters.

In [ ]:
def ensure_root_dir_on_path():
    import os
    import sys

    sources_root_path = os.path.abspath("../src")
    if sources_root_path not in sys.path:
        sys.path.append(sources_root_path)


ensure_root_dir_on_path()

In [ ]:
import torch

from decoupledbo.modules.gp_testproblem import (
    # GPTestProblem,
    bounds_to_tensor,
    create_gp_problem_model,
    estimate_reference_point_and_hypervolume,
)
from decoupledbo.pipeline.data_catalog import DataCatalog
from tqdm.notebook import tqdm

## Description

In [ ]:
n_repeats = 100

bounds = [(0, 1), (0, 1)]

# Family: Objectives with different length scales
master_seed = 14101066
problem_name = "lengthscales"
hyperparams = {"length_scales": [0.2, 1.8], "output_scales": [1, 50], "means": [0, 0]}

# Family: Objectives with and without observation noise
# master_seed = 25101415
# problem_name = "observationnoise"
# hyperparams = {"length_scales": [0.4, 0.4], "means": [0, 0], "output_scales": [1, 1]}

## Generation

In [ ]:
torch.manual_seed(master_seed)
seeds = torch.randint(10_000_000, (n_repeats, 3))

In [ ]:
def generate_and_save_problem(problem_name, i, bounds, hyperparams, seeds):
    """ Generate a test problem and save the result in the data catalog
    
    Args:
        problem_name: The problem will be saved at `problem_name/i.pt`
        i: The index to give to the problem
        bounds: A list whose jth element is a 2-tuple containing the lower and
            upper bounds for the jth objective
        hyperparams: A dictionary of hyperparameters (length scales, output
            scales and means for each objective).
        seeds: A list of integers containing the random seeds to use to generate
            problem (1+n seeds are required where n is the number of objectives)
    """
    problem_model, train_x, train_y = create_gp_problem_model(
        bounds,
        n_objectives=len(bounds),
        **hyperparams,
        input_seed=seeds[0].item(),
        output_seeds=seeds[1:].tolist(),
        dtype=torch.double,
    )
    ref_point, hv = estimate_reference_point_and_hypervolume(
        problem_model, bounds_to_tensor(bounds), dtype=torch.double
    )
    # problem = GPTestProblem(
    #     problem_model,
    #     bounds=bounds,
    #     ref_point=ref_point.tolist(),
    #     max_hv=float(hv),
    # )
    
    DataCatalog.save_shared_gp_test_problem_data(
        name=f"{problem_name}/{i}",
        bounds=bounds,
        fixed_hyperparams=hyperparams,
        model_state_dict=problem_model.state_dict(),
        train_x=train_x,
        train_y=train_y,
        ref_point=ref_point.tolist(),
        max_hv=float(hv),
        negate=False,
    )

In [ ]:
for i in tqdm(range(n_repeats), desc="Generating test problems"):
    generate_and_save_problem(problem_name, i, bounds, hyperparams, seeds[i])